In [ ]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}
.code_block {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-size: 75%;
    line-height: 22px; /* 5px +12px + 5px */
    #text-indent: 25px;
    #background-color: #fbfbea;
    padding: 5px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h2>
<center>
Building a Bag a Words with Spark
</center>
</h2>

<div class=h1_cell>
<p>
This notebook assumes you have already ran and understand the code in setup_spark.ipynb. Lets connect to our already installed spark cluster.
</div>

In [1]:
import os
import sys
import subprocess

In [2]:
os.environ['SPARK_HOME'] = os.environ['HOME'] + '/spark'
os.environ['PATH'] += ':' + os.environ['SPARK_HOME'] + '/bin'
sys.path.append(os.environ['SPARK_HOME'] + '/python')
sys.path.append(os.environ['SPARK_HOME'] + '/python/lib/py4j-0.10.6-src.zip')

In [3]:
# run start-all.sh
subprocess.call(os.environ['SPARK_HOME'] + "/sbin/start-all.sh", env=os.environ)

0

<div class=h1_cell>
<p>
Lets first configure the spark environment start the spark jvm application. We can then set some of spark's cluster settings like <i>spark.executor.memory</i>, which controls how much RAM an spark worker process gets. Finally, we can connect to the spark app and get our spark session object.
</div>

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().set('spark.executor.cores', 1).set('spark.executor.memory', '6g')
spark = SparkSession(SparkContext(master='spark://azure4:7077', conf=conf))
spark

<h3>
<center>
Why Spark?
</center>
</h3>

<div class=h1_cell>
<p>
This notebook is a walkthrough of using Spark to build a bag of words from an collection of essays. Spark has 2 core data structures that we operate on: RDD's and DataFrames, corresponding to low and high level libraries respectively. We explore both low-level and high-level Spark implementations.
<p>
A spark DataFrame is a distributed collection of data grouped into named columns. Notice how this differs from Pandas: <strong>it is not indexed by row</strong>. DataFrames are also strongly typed, either infering the schema from the data or being passed one. This allows spark to bypass type inference later when performing transformations, a costly operation in python. We can still access columns by name, however.
<p>
<strong>Using DataFrames:</strong>
<ol>
    <li>Load dataset from a pandas dataframe to a spark dataframe.</li>
    <li>Build a bag of words for each row's (project's) text</li>
    <li>Sum the counts of matching (word, project approval) keys</li>
    <li>Combine the counts of each project approval by (word) keys</li>
</ol>
<p>
A Spark RDD (Resilient Distributed Datasets) is a immutable, partitioned collection of elements that can be operated on in parallel. DataFrames are built on top of RDD's. RDD's do not have column attributes, but can be operated on using the popular map-reduce pattern. We access our RDD from the DataFrame structre we worked with before, allowing us to still access column attributes when writing map or reduce functions.
<p>
<strong>Using RDD's:</strong>
<ol>
    <li>We'll break the dataframe into two subtables: rows with label value 0 and rows with label value 1.</li>
    <li>We'll distribute each subtable to two nodes. Each will work on its own set of rows, producing a (word, count) dictionary.</li>
    <li>We'll combine the counts of matching words into an array.</li>
</ol>
<p>
We'll walk though each step with DataFrames, and will outline using using RDD's in greater detail below.
<p>
Each data structure has its own advantages, but Spark recommends we work with its higher level DataFrames in Pyspark. Spark even provides a higher level data structure in Scala called DataSets. This is because the enforcement of a schema and indexed columns give dataframe operations space and performance gains. However in Pyspark, RDD's give us control that we lack with DataFrames, and it is helpful to understand what is happening under the hood.
</div>

<div class=h1_cell>
<p>
Lets get the data.
</div>

In [4]:
import pandas as pd
url = 'https://www.dropbox.com/s/2hdbltrl8bh6kbu/train.csv?raw=1'
donate_table = pd.read_csv(url, encoding='utf-8')

In [3]:
len(donate_table.index)

182080

<div class=h1_cell>
<p>
Lets get a subset of the columns.
</div>

In [5]:
donate_table = donate_table[['project_essay_1', 'project_essay_2', 'project_title', 'project_is_approved']]
donate_table.head(5)

,project_essay_1,project_essay_2,project_title,project_is_approved
0,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,Super Sight Word Centers,1
1,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,Keep Calm and Dance On,0
2,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,Lets 3Doodle to Learn,1
3,My students are the greatest students but are ...,"The student's project which is totally \""kid-i...","\""Kid Inspired\"" Equipment to Increase Activit...",0
4,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,We need clean water for our culinary arts class!,1


In [6]:
import sys
sys.getsizeof(donate_table)

1148460448

<h3>
<center>
1. Load dataset from a pandas dataframe to a spark dataframe.
</center>
</h3>

<div class=h1_cell>
<p>
We now have the pandas version of our dataset. We could just pass the dataframe to spark, but spark performs type inference on each value of the dataset to generate a schema if we don't give it one ourselves. So lets create a schema object, and tell spark not to check for itself.
</div>

In [6]:
from pyspark.sql.types import ByteType, StringType, StructType, StructField

schema = StructType([StructField('project_essay_1', StringType()),
                     StructField('project_essay_2', StringType()),
                     StructField('project_title', StringType()),
                     StructField('project_is_approved', ByteType())])

dtdf = spark.createDataFrame(donate_table, schema=schema, verifySchema=False)
dtdf.show(5)

+--------------------+--------------------+--------------------+-------------------+
|     project_essay_1|     project_essay_2|       project_title|project_is_approved|
+--------------------+--------------------+--------------------+-------------------+
|Most of my kinder...|I currently have ...|Super Sight Word ...|                  1|
|Our elementary sc...|We strive to prov...|Keep Calm and Dan...|                  0|
|Hello;\r\nMy name...|We are looking to...|Lets 3Doodle to L...|                  1|
|My students are t...|The student's pro...|\"Kid Inspired\" ...|                  0|
|My students are a...|For some reason i...|We need clean wat...|                  1|
+--------------------+--------------------+--------------------+-------------------+
only showing top 5 rows



<div class=h1_cell>
<p>
Lets combine the text of all the columns.
</div>

In [7]:
from pyspark.sql.functions import concat_ws

dtdf = dtdf.withColumn('text', concat_ws(' ', dtdf.project_title, dtdf.project_essay_1, dtdf.project_essay_2))
dtdf = dtdf.drop(*['project_essay_1', 'project_essay_2', 'project_title'])
dtdf.show(5)

+-------------------+--------------------+
|project_is_approved|                text|
+-------------------+--------------------+
|                  1|Super Sight Word ...|
|                  0|Keep Calm and Dan...|
|                  1|Lets 3Doodle to L...|
|                  0|\"Kid Inspired\" ...|
|                  1|We need clean wat...|
+-------------------+--------------------+
only showing top 5 rows



<h3>
<center>
2. Build a bag of words for each row's (project's) text
</center>
</h3>

<div class=h1_cell>
<p>
We can break each project's text into an array of significant words. But how do we identify when a word is not significant? We can use nltk to tokenize sentences (split into words) and use its stopwords list to filter out insignificant words.
<p>
We also have to remember that this function will be distributed across the cluster and be performed row by row. We don't want to create a WordPuncTokenizer and download the stopwords for every row in the dataset, so we're going to create them once and broadcast them across the cluster, so each worker node has its own copy.
<p>
After we create an array of words for each project, we'll create a bag of words for each project, counting how many times each word was used in the list.
<p>
Make sure nltk is installed on each node in the cluster.
</div>

In [ ]:
# If nltk is not install on worker nodes
!ssh instance-7 'pip install nltk'
!ssh instance-8 'pip install nltk'

In [ ]:
!pip install nltk

In [8]:
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk import download

from string import punctuation

tokenizer = WordPunctTokenizer()
download('stopwords')
swords = stopwords.words('english')

# Broadcasting across cluster
b_tokenizer = spark.sparkContext.broadcast(WordPunctTokenizer())
b_swords = spark.sparkContext.broadcast(swords)

def sentence_wrangler(text):
    word_list = b_tokenizer.value.tokenize(text.lower())
    result = []
    for word in word_list:
        if word in b_swords.value:
            continue
        check = False
        for char in word:
            if char in punctuation:
                check = True
                break
        if not check: result.append(word)
    return result

def build_bag(words):
    d = {}
    for word in words:
        if word in d:
            d[word] += 1
        else:
            d[word] = 1
    return d

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jakeu123/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType

sentence_wrangler_udf = udf(sentence_wrangler, ArrayType(StringType()))
dtdf = dtdf.withColumn('words', sentence_wrangler_udf('text'))
dtdf.show(5)

+-------------------+--------------------+--------------------+
|project_is_approved|                text|               words|
+-------------------+--------------------+--------------------+
|                  1|Super Sight Word ...|[super, sight, wo...|
|                  0|Keep Calm and Dan...|[keep, calm, danc...|
|                  1|Lets 3Doodle to L...|[lets, 3doodle, l...|
|                  0|\"Kid Inspired\" ...|[kid, inspired, e...|
|                  1|We need clean wat...|[need, clean, wat...|
+-------------------+--------------------+--------------------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, IntegerType


build_bag_udf = udf(build_bag, MapType(StringType(), IntegerType()))
dtdf = dtdf.withColumn('bag', build_bag_udf('words'))
dtdf = dtdf.drop('text', 'words')
dtdf.cache()
dtdf.show(5)

+-------------------+--------------------+
|project_is_approved|                 bag|
+-------------------+--------------------+
|                  1|[practice -> 3, d...|
|                  0|[temporary -> 1, ...|
|                  1|[88 -> 1, big -> ...|
|                  0|[00 -> 1, educati...|
|                  1|[reason -> 1, fee...|
+-------------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import concat_ws, to_json

dtdf_str = dtdf.withColumn('freqMap', to_json(dtdf.bag))
dtdf.unpersist()
dtdf_str.cache()
dtdf_str.show(5)

In [29]:
#bag = dtdf_str.drop('bag')
#bag.cache()
#dtdf_str.unpersist()
#pd_bag = bag.toPandas()
pd_bag.to_csv('bag.csv', encoding='utf-8')
del pd_bag

<h3>
<center>
3. Sum the counts of matching (word, project approval) keys
</center>
</h3>

<div class=h1_cell>
<p>
The explode creates a dataframe by making a row of each key, val pair in the map structure passed to it. We then group by word and project_is_approved, adding the counts of all matching entries.
</div>

In [11]:
from pyspark.sql.functions import explode

bagdf = dtdf.select('project_is_approved', explode(dtdf.bag).alias('word', 'count'))
bagdf.cache()
dtdf.unpersist()
groupby_df = bagdf.groupBy('word', 'project_is_approved').agg({'count': 'sum'})
groupby_df.cache()
bagdf.unpersist()
groupby_df.show(5)

+----------+-------------------+----------+
|      word|project_is_approved|sum(count)|
+----------+-------------------+----------+
|      last|                  1|     11566|
|       buy|                  1|      3037|
|accomplish|                  1|      3465|
|   qualify|                  1|      6818|
|  teaching|                  0|      4905|
+----------+-------------------+----------+
only showing top 5 rows



<div class=h1_cell>
<p>
Install pyarrow and pandas across the cluster if you haven't.
</div>

In [ ]:
!pip install pyarrow==0.9.*
#!ssh instance-7 'pip install pyarrow==0.9.*'
#!ssh instance-8 'pip install pyarrow==0.9.*'

In [ ]:
!ssh instance-7 'pip install pandas'
!ssh instance-8 'pip install pandas'

<h3>
<center>
4. Combine the counts of each project approval by (word) keys
</center>
</h3>

<div class=h1_cell>
<p>
Now lets groupBy each word again, which will create pandas dataframes containing either 1 or 2 rows. We'll transform this dataframe to a single-row dataframe with the word and its frequency with each project approval value as columns.
</div>

In [12]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import LongType

schema = StructType([StructField('word', StringType()),
                     StructField('counts', ArrayType(LongType()))])

def combine_sums(pdf):
    counts = [0, 0]
    if len(pdf.index) == 1:
        idx = pdf.loc[0].project_is_approved
        counts[idx] = pdf.loc[0]["sum(count)"]
    else:
        if pdf.loc[0].project_is_approved == 0:
            counts[0] = pdf.loc[0]["sum(count)"]
            counts[1] = pdf.loc[1]["sum(count)"]
        else:
            counts[0] = pdf.loc[1]["sum(count)"]
            counts[1] = pdf.loc[0]["sum(count)"]
    return pd.DataFrame([[pdf.loc[0].word, counts]], columns=['word', 'counts'])

cs_udf = pandas_udf(combine_sums, schema, PandasUDFType.GROUPED_MAP)

bagdf = groupby_df.groupBy('word').apply(cs_udf)

bagdf.show(5)  # 4 Minutes for 100,000 projects

+-----+------+
| word|counts|
+-----+------+
|   07|[3, 7]|
|14x20|[0, 1]|
|1970s|[1, 6]|
|  296|[0, 3]|
|   3x|[2, 7]|
+-----+------+
only showing top 5 rows



In [13]:
bagdf.cache()
groupby_df.unpersist()
bagdf.sample(fraction=0.05).show()

+-----------+-------------+
|       word|       counts|
+-----------+-------------+
|      anime|      [5, 60]|
|        art|[6862, 30682]|
|     biting|      [7, 21]|
|   cropland|       [0, 1]|
|     ecause|       [0, 1]|
|       ells|   [103, 449]|
|       holz|       [1, 3]|
|       iffy|       [0, 1]|
|    jeebies|       [0, 1]|
| kleenslate|      [0, 17]|
|   mindmash|       [0, 1]|
|nexperience|       [3, 8]|
| paramedics|       [0, 4]|
|     prople|       [1, 0]|
|reinfornced|       [0, 1]|
|    theough|       [0, 1]|
|     voyage|      [2, 28]|
| wonderfuls|       [0, 1]|
|          ্|       [0, 1]|
|    allowes|       [0, 1]|
+-----------+-------------+
only showing top 20 rows



In [19]:
bagdf.toPandas().to_csv('bag.csv', encoding='utf-8')

In [16]:
bagdf.unpersist()

DataFrame[word: string, counts: array<bigint>]

<div class=h1_cell>
<p>
This isn't much faster than the local version, not to mention we don't have the space capacity to compute the entire dataset. The reality is that Spark's advantages are at scale, when workloads are distributed across anywhere from tens to hundreds of nodes. When you have 6 CPU's and have to perform 2 separate groupby operations, Spark's advantages are harder to see.
<p>
Spark's higher-level DataFrame API provides some optimizations, but doesn't allow for the flexibility of working directly with RDD's. Lets see if we can get rid of one of those groupby's by splitting the dataset across the cluster into 2 separate RDD's based on the value of project_is_approved, applying the groupby to each partition and re-joining the 2 RDD's by combining the counts from both partitions to an array.
</div>

In [14]:
from pyspark.sql.functions import explode

freqdf = dtdf.select('project_is_approved', explode(dtdf.bag).alias('word', 'count'))
freqdf.cache()
freqdf.show(5)

+-------------------+--------------+-----+
|project_is_approved|          word|count|
+-------------------+--------------+-----+
|                  1|      practice|    3|
|                  1|differentiated|    1|
|                  1|          year|    3|
|                  1|    considered|    1|
|                  1|     obstacles|    1|
+-------------------+--------------+-----+
only showing top 5 rows



In [15]:
def groupBy(iterator):
    d = {}
    for row in iterator:
        if row[1].word in d:
            d[row[1].word] += 1
        else:
            d[row[1].word] = 1
    yield d.items()

<div class=h1_cell>
<p>
Per line:
<ol>
    <li>create key-value pairs of the rows</li>
    <li>split the dataset by key, with one piece on each node</li>
    <li>apply groupBy to each partition</li>
    <li>re-combine into (word, (count, approval))</li>
    <li>combine key (word) pairs and their counts for each approval value</li>
    <li>Get the first x</li>
</ol>
</div>

In [17]:
bag = freqdf.rdd.map(lambda x: (x.project_is_approved, x))\
            .partitionBy(2, lambda x: int(x))\
            .mapPartitions(groupBy)\
            .zipWithIndex().flatMap(lambda x: [(y[0], (y[1], x[1])) for y in x[0]])\
            .reduceByKey(lambda x, y: [x[0], y[0]])\
            .take(20)

In [18]:
bag

[(u'fawn', [1, 1]),
 (u'refreshable', (1, 1)),
 (u'preparesee', (1, 1)),
 (u'tah', (1, 1)),
 (u'comically', (1, 1)),
 (u'localized', (2, 1)),
 (u'sevens', (2, 1)),
 (u'saylor', (1, 1)),
 (u'sprague', (2, 1)),
 (u'chatter', [15, 97]),
 (u'lence', (1, 0)),
 (u'originality', [11, 29]),
 (u'alphabetimals', (1, 0)),
 (u'gaa', (2, 0)),
 (u'searat', (1, 1)),
 (u'nreading', [148, 918]),
 (u'capoeira', (2, 1)),
 (u'potentiometers', (1, 1)),
 (u'compuesta', (1, 1)),
 (u'politician', [1, 11])]

<div class=h1_cell>
<p>
You can see that not all rows have the same format. This is because only a subset of words appeared in essays with both approval values. Those that were only in essays that were approved or denied still have the (word, (count, approval)) format.
<p>
While I could explore how to fix this, the reality is that this is slower than the previous strategy. I believe this is a result of the fact that we iterate over the dataset and split it into 2 unequal partitions. It is possible CPU resources are being wasted. Our idea of avoiding 2 groupBy's also barely happens because we have to combine the word pairs in the end with a reduceByKey.
<p>
If you want to give up the free credits and create a larger cluster on Google Cloud, be my guest. Spark is used everywhere in the industry, so I want to see the performance benefits of the tool. For now, I hope the examples within each notebook have gotten you familiar with Spark.
</div>

In [17]:
spark.stop()

In [23]:
subprocess.call(os.environ['SPARK_HOME'] + "/sbin/stop-all.sh", env=os.environ)

0